In [1]:
import csv
import torch
import numpy as np
import pandas as pd

fields = ['input1', 'input2','output1', 'output2']
rows = []
h_target = torch.tensor([[1.,1.],[1.,-1]] / np.sqrt(2), dtype=torch.complex128)

for i in range(8):
    phi = np.random.rand() * 2*np.pi
    theta = np.random.rand() * np.pi
    
    input = torch.tensor([np.sin(theta), np.cos(theta) * np.exp(1j * phi)])
    output = h_target @ input

    rows.append([np.sin(theta), np.cos(theta) * np.exp(1j * phi), output[0].item(), output[1].item()])

filename = 'qubit.csv'

with open(filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields)
    csvwriter.writerows(rows)

In [2]:
df = pd.read_csv('qubit.csv')
df

,input1,input2,output1,output2
0,0.530280,(0.47692819582453877-0.7009586668398214j),(0.7122034553131572-0.4956526266539196j),(0.03772513249996323+0.4956526266539196j)
1,0.761031,(-0.07806590657159748+0.6440013848383618j),(0.48292913161738243+0.45537774631273303j),(0.5933309954498865-0.45537774631273303j)
2,0.948814,(-0.26858914850895893-0.16616662533815532j),(0.4809917972469542-0.11749754758349401j),(0.8608342137745653+0.11749754758349401j)
3,0.337110,(-0.617552802823716-0.7106230056734187j),(-0.19830266420827636-0.5024863461788408j),(0.6750488850265406+0.5024863461788408j)
4,0.062857,(-0.7093891774057459+0.7020084986338084j),(-0.45716694729891544+0.49639496983455306j),(0.546060848388984-0.49639496983455306j)
5,0.608016,(-0.4777472599386458-0.634092874315152j),(0.09211409910144835-0.4483713713303132j),(0.7677507534932657+0.4483713713303132j)
6,0.999684,(-0.02363105329328449-0.008563392458402899j),(0.6901737075538995-0.006055232877298429j),(0.7235930636144238+0.006055232877298429j)
7,0.356960,(-0.8039760399866672-0.47560685281455456j),(-0.3160878680668973-0.33630483080396373j),(0.8209059515052612+0.33630483080396373j)


In [21]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset
from torch.nn.parameter import Parameter

num_qubits = 8

num_epochs = 1000

learning_rate = 1e-4

class input_vec_dataset(Dataset):
    def __init__(self):
        
        self.df = pd.read_csv('qubit.csv')
        self.df['input1'] = self.df['input1'].astype(complex)
        self.df['input2'] = self.df['input2'].astype(complex)
        self.df['output1'] = self.df['output1'].astype(complex)
        self.df['output2'] = self.df['output2'].astype(complex)

        dataset = []
        labels = []

        for i in range(num_qubits):
            dataset.append([self.df['input1'][i],self.df['input2'][i]])
            
        self.dataset = torch.tensor(dataset, dtype=torch.complex128)
        
        for i in range(num_qubits):
            labels.append([self.df['output1'][i],self.df['output2'][i]])
        
        self.labels = torch.tensor(labels, dtype=torch.complex128)
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx],self.labels[idx]

data_set = input_vec_dataset()

data_loader = torch.utils.data.DataLoader(data_set, batch_size=1, shuffle=False, drop_last=False)

class HModel(torch.nn.Module):
    
    def __init__(self):
        super(HModel, self).__init__()
#         self.θ = Parameter(torch.rand(1))
#         self.α = Parameter(torch.rand(1))
#         self.β = Parameter(torch.rand(1))
#         self.ϕ = Parameter(torch.rand(1))
#         self.U = torch.exp(1j * self.ϕ / 2) * torch.tensor(
#              [[torch.exp(1j * self.α) * torch.cos(self.θ), torch.exp(1j * self.β) * torch.sin(self.θ)],
#              [- torch.exp(-1j * self.β) * torch.sin(self.θ), torch.exp(-1j * self.α) * torch.cos(self.θ)]], 
#              dtype=torch.complex128, requires_grad = True)
#         self.U = Parameter(torch.tensor([[1,1],[1,1]], dtype=torch.complex128, requires_grad=True))
        self.U = Parameter(torch.rand(2,2, dtype=torch.complex128))
        
    def forward(self, x):
#         θ = self.θ
#         α = self.α
#         β = self.β
#         ϕ = self.ϕ
        U = self.U
#         U = torch.exp(1j * ϕ / 2) * torch.tensor(
#             [[torch.exp(1j * α) * torch.cos(θ), torch.exp(1j * β) * torch.sin(θ)],
#              [- torch.exp(-1j * β) * torch.sin(θ), torch.exp(-1j * α) * torch.cos(θ)]], 
#             dtype=torch.complex128)
        
        x = torch.matmul(x,U)

        return x
    
model = HModel()

# class CustomLoss(nn.Module):
#     def __init__(self):
#         super(CustomLoss, self).__init__()

#     def forward(self, inputs, targets):
# #         loss = 1 - (abs(torch.dot(torch.conj(targets).reshape(-1),inputs.reshape(-1))))**2
# #         loss = 1 - (abs(torch.conj(labels).reshape(-1) @ batch.reshape(-1)))**2
# #         loss = 1 - (abs((torch.conj(targets) * inputs).sum(dim=-1)))**2
#         loss = abs((targets - inputs)**2)
#         return loss.mean()


def get_states_1d(batch, num_qubits_in_batch = 1):
        """Return the states in a 1d tensor."""
        num_states_in_batch = batch.shape[0]
        return torch.reshape(batch, [num_states_in_batch, 2**num_qubits_in_batch])
    
def CustomLoss(inputs, targets):
#     loss = torch.mean(abs(1 - (abs((torch.conj(targets) * inputs).sum(dim=-1)))**2))
#     loss = 1 - torch.dot(labels, inputs).abs() ** 2
    losses = [1 - torch.dot(get_states_1d(t), get_states_1d(i)).abs() ** 2 for t, i in zip(targets,inputs)]
    
#     loss = 1 - torch.dot(get_states_1d(targets), get_states_1d(inputs)).abs() ** 2
    return torch.mean(losses)
    
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    
    for batch, labels in data_loader:
         
        if epoch == 0:
#             print(batch)
#             print(labels)
            print([(b,l) for b,l in zip(batch,labels)])
        optimizer.zero_grad()
        
        outputs = model(batch)
        
        loss = CustomLoss(outputs,labels)
#         loss = criterion(abs(outputs), abs(labels))
        
        loss.backward(retain_graph=True)
        
        optimizer.step()
        
    if epoch % 100 == 0:
        
        print(f'epoch: {epoch}, loss: {loss}')
        
model.eval()

with torch.no_grad():
    out_data = model(torch.tensor([0,1],dtype=torch.complex128))
    print(out_data)
#     print(np.conj(out_data) * out_data) 
    print(torch.abs(out_data))

[(tensor([0.5303+0.0000j, 0.4769-0.7010j], dtype=torch.complex128), tensor([0.7122-0.4957j, 0.0377+0.4957j], dtype=torch.complex128))]


RuntimeError: shape '[2, 2]' is invalid for input of size 2

In [10]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset

num_qubits = 8

num_epochs = 100

learning_rate = 0.01

class input_vec_dataset(Dataset):
    def __init__(self):
        
        self.df = pd.read_csv('qubit.csv')
        self.df['input1'] = self.df['input1'].astype(complex)
        self.df['input2'] = self.df['input2'].astype(complex)
        self.df['output1'] = self.df['output1'].astype(complex)
        self.df['output2'] = self.df['output2'].astype(complex)

        dataset = []
        labels = []

        for i in range(num_qubits):
            dataset.append([self.df['input1'][i],self.df['input2'][i]])
            
        self.dataset = torch.tensor(dataset, dtype=torch.complex128)
        
        for i in range(num_qubits):
            labels.append([self.df['output1'][i],self.df['output2'][i]])
        
        self.labels = torch.tensor(labels, dtype=torch.complex128)
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx],self.labels[idx]

data_set = input_vec_dataset()

data_loader = torch.utils.data.DataLoader(data_set, batch_size=1, shuffle=True, drop_last=False)

class HModel(torch.nn.Module):
    
    def __init__(self):
        super(HModel, self).__init__()     
        self.linear = nn.Linear(2,2,bias=False, dtype=torch.complex128)
        
    def forward(self, x):
        x = self.linear(x)
        return x
    
model = HModel()

def CustomLoss(inputs, targets):
    loss = torch.mean(abs(1 - (abs((torch.conj(targets) * inputs).sum(dim=-1)))**2))
    loss.requires_grad=True
    return loss

# class CustomLoss(nn.Module):
#     def __init__(self):
#         super(CustomLoss, self).__init__()

#     def forward(self, inputs, targets):
#         loss = inputs - targets
#         return loss.mean()
    
# loss_fn = CustomLoss()
# criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    
    for batch, labels in data_loader:
 
        optimizer.zero_grad()
        
        outputs = model(batch)
        
        loss = CustomLoss(batch,labels)
        
        loss.backward()
        
        optimizer.step()
        
    if epoch % 10 == 0:
        
        print(f'epoch: {epoch}, loss: {loss}')
        
model.eval()

with torch.no_grad():

    out_data = model(torch.tensor([0+0j,1+0j],dtype=torch.complex128))
    print(out_data)
#     print(np.conj(out_data) * out_data) 
    print(torch.abs(out_data))

epoch: 0, loss: 0.9994144162415406
epoch: 10, loss: 0.9994144162415406
epoch: 20, loss: 0.5242399086635763
epoch: 30, loss: 0.9803126604981304
epoch: 40, loss: 0.5259624880361987
epoch: 50, loss: 0.48980423733305134
epoch: 60, loss: 0.9994144162415406
epoch: 70, loss: 0.6004223450815089
epoch: 80, loss: 0.9808413089479036
epoch: 90, loss: 0.5242399086635763
tensor([ 0.6097+0.1286j, -0.1190-0.5503j], dtype=torch.complex128)
tensor([0.6231, 0.5630], dtype=torch.float64)


In [22]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import Dataset
from torch.nn.parameter import Parameter

num_qubits = 8

num_epochs = 1000

learning_rate = 1e-3

class input_vec_dataset(Dataset):
    def __init__(self):
        
        self.df = pd.read_csv('qubit.csv')
        self.df['input1'] = self.df['input1'].astype(complex)
        self.df['input2'] = self.df['input2'].astype(complex)
        self.df['output1'] = self.df['output1'].astype(complex)
        self.df['output2'] = self.df['output2'].astype(complex)

        dataset = []
        labels = []

        for i in range(num_qubits):
            dataset.append([self.df['input1'][i],self.df['input2'][i]])
            
        self.dataset = torch.tensor(dataset, dtype=torch.complex128)
        
        for i in range(num_qubits):
            labels.append([self.df['output1'][i],self.df['output2'][i]])
        
        self.labels = torch.tensor(labels, dtype=torch.complex128)
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx],self.labels[idx]

data_set = input_vec_dataset()

data_loader = torch.utils.data.DataLoader(data_set, batch_size=1, shuffle=False, drop_last=False)

class HModel(torch.nn.Module):
    
    def __init__(self):
        super(HModel, self).__init__()
        self.θ = Parameter(torch.rand(1))
        self.α = Parameter(torch.rand(1))
        self.β = Parameter(torch.rand(1))
        self.ϕ = Parameter(torch.rand(1))
        self.U = Parameter(torch.exp(1j * self.ϕ / 2) * torch.tensor(
             [[torch.exp(1j * self.α) * torch.cos(self.θ), torch.exp(1j * self.β) * torch.sin(self.θ)],
             [- torch.exp(-1j * self.β) * torch.sin(self.θ), torch.exp(-1j * self.α) * torch.cos(self.θ)]], 
             dtype=torch.complex128, requires_grad = True))
#         self.U = Parameter(torch.tensor([[1,1],[1,1]], dtype=torch.complex128, requires_grad=True))
#         self.U = Parameter(torch.rand(2,2, dtype=torch.complex128))
        
    def forward(self, x):
        θ = self.θ
        α = self.α
        β = self.β
        ϕ = self.ϕ
        U = self.U
#         U = torch.exp(1j * ϕ / 2) * torch.tensor(
#             [[torch.exp(1j * α) * torch.cos(θ), torch.exp(1j * β) * torch.sin(θ)],
#              [- torch.exp(-1j * β) * torch.sin(θ), torch.exp(-1j * α) * torch.cos(θ)]], 
#             dtype=torch.complex128)
        
        x = torch.matmul(x,U)

        return x
    
model = HModel()

# class CustomLoss(nn.Module):
#     def __init__(self):
#         super(CustomLoss, self).__init__()

#     def forward(self, inputs, targets):
# #         loss = 1 - (abs(torch.dot(torch.conj(targets).reshape(-1),inputs.reshape(-1))))**2
# #         loss = 1 - (abs(torch.conj(labels).reshape(-1) @ batch.reshape(-1)))**2
# #         loss = 1 - (abs((torch.conj(targets) * inputs).sum(dim=-1)))**2
#         loss = abs((targets - inputs)**2)
#         return loss.mean()
    
def CustomLoss(inputs, targets):
    loss = torch.mean(abs(1 - (abs((torch.conj(targets) * inputs).sum(dim=-1)))**2))
    return loss
    
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    
    for batch, labels in data_loader:
 
        optimizer.zero_grad()
        
        outputs = model(batch)
        
        loss = CustomLoss(outputs,labels)
#         loss = criterion(abs(outputs), abs(labels))
        
        loss.backward(retain_graph=True)
        
        optimizer.step()
        
    if epoch % 100 == 0:
        
        print(f'epoch: {epoch}, loss: {loss}')
        
model.eval()

with torch.no_grad():
    out_data = model(torch.tensor([0,1],dtype=torch.complex128))
    print(out_data)
#     print(np.conj(out_data) * out_data) 
    print(torch.abs(out_data))

epoch: 0, loss: 0.5029782558219336
epoch: 100, loss: 0.14273499138495238
epoch: 200, loss: 0.001067397537366399
epoch: 300, loss: 0.0006012623583673271
epoch: 400, loss: 0.0007161919482363555
epoch: 500, loss: 0.0011347462323046953
epoch: 600, loss: 0.0001832807391004554
epoch: 700, loss: 0.00428404950107153
epoch: 800, loss: 0.00010701472914331767
epoch: 900, loss: 0.0010896887842839487
tensor([-0.6600+0.4493j,  0.4413-0.4643j], dtype=torch.complex128)
tensor([0.7984, 0.6406], dtype=torch.float64)
